# Sentiment Analysis - Lyrics Extraction

- Website: https://genius.com/

- Genius API client: https://genius.com/developers?ref=genius.engineering


**Extract Lyrics**
- requests module
- create an .env file to store you API token 
- use python-dotenv package to load the env variable from the .env file
- install in conda environment:
    - conda install -c conda-forge python-dotenv
- install in vs code terminal:
    - pip install python-dotenv


In [3]:
import os 
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup

#load environment variables from .env file
load_dotenv()

# genius API token from environment variable 
TOKEN = os.getenv('GENIUS_API_TOKEN')
BASE_URL = "https://api.genius.com"

# function to search for a song
def search_song(song_title, artist_name):
    search_url = f'{BASE_URL}/search'
    headers = {'Authorization':f"Bearer {TOKEN}"}
    params = {'q': f"{song_title} {artist_name}"}
    
    response = requests.get(search_url, headers=headers, params=params)
    response_json= response.json()
    song_hit = response_json['response']['hits'][0]['result']
    
    return song_hit['id']

# function to get song lyrics
def get_lyrics(song_id):
    song_url = f'{BASE_URL}/songs/{song_id}'
    headers = {'Authorization': f'Bearer {TOKEN}'}
    
    response = requests.get(song_url, headers=headers)
    song_info = response.json()['response']['song']
    lyrics_path = song_info['path']
    
    lyrics_url = f'https://genius.com{lyrics_path}'
    lyrics_response = requests.get(lyrics_url)
    lyrics_html = lyrics_response.text
    
    # extract lyrics from HTML
    soup = BeautifulSoup(lyrics_html, 'html.parser')
    
     # Genius lyrics are usually inside <div> tags with class "Lyrics__Container"
    lyrics = ""
    for div in soup.find_all("div", class_=lambda x: x and 'Lyrics__Container' in x):
        lyrics += div.get_text(separator="\n")
    
    return lyrics if lyrics else "Lyrics not found"
    
song_title = 'How Deep Is Your Love'
artist_name = 'Bee Gees'
song_id = search_song(song_title, artist_name)
lyrics = get_lyrics(song_id)

lyrics

KeyError: 'response'